In [ ]:
import json
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

In [ ]:
f = open('../data/sim-vs-testnet/testnet-results-169-1.json')
data = json.load(f)
df_test = pd.json_normalize(data)
f.close()

f = open('../data/sim-vs-testnet/sim-results-169-5.json')
data = json.load(f)
df_sim = pd.json_normalize(data)
f.close()

In [ ]:
df_test['marketCap'] = df_test['marketCap'].map(lambda x: x/1e18)
df_test['price'] = df_test['price'].map(lambda x: round(x/1e18,2))
df_test['lowWall'] = df_test['lowWall'].map(lambda x: round(x/1e18,2))
df_test['highWall'] = df_test['highWall'].map(lambda x: round(x/1e18,2))
df_test['lowCushion'] = df_test['lowCushion'].map(lambda x: round(x/1e18,2))
df_test['highCushion'] = df_test['highCushion'].map(lambda x: round(x/1e18,2))
df_test['lowCapacity'] = df_test['lowCapacity'].map(lambda x: x/1e18)
df_test['highCapacity'] = df_test['highCapacity'].map(lambda x: x/1e9) * df_test['highWall']
df_test['reserves'] = df_test['reserves'].map(lambda x: x/1e18)
df_test['liqRatio'] = df_test['liqRatio'].map(lambda x: x/10000)
df_test['supply'] = df_test['supply'].map(lambda x: x/1e9)
df_test['day'] = df_test['epoch'].map(lambda x: x//3)

df_test[19:].head()

rows = [i for i in range (1, 1095)]
keep = [i for i in range (0, 1095, 3)]
drop = list(set(rows) - set(keep))

df_testnet = df_test.drop(df_test.index[drop])
df_testnet = df_testnet.set_index(['day'])
df_testnet = df_testnet.drop(columns=['key','epoch','seed'])

df_testnet2 = df_testnet[1:]
df_testnet2 = df_testnet2.reset_index()
df_testnet2['day'] = df_testnet2['day'].map(lambda x: x-1)

df_testnet2.head()

In [ ]:
df_sim = df_sim.drop(columns=['seed','day'])
df_sim['price'] = df_sim['price'].apply(lambda x: round(float(x),2))
df_sim['netFlow'] = df_sim['netFlow'].apply(lambda x: round(float(x),2))
df_sim['lowCushion'] = df_sim['lowCushion'].apply(lambda x: round(float(x),2))
df_sim['lowWall'] = df_sim['lowWall'].apply(lambda x: round(float(x),2))
df_sim['highCushion'] = df_sim['highCushion'].apply(lambda x: round(float(x),2))
df_sim['highWall'] = df_sim['highWall'].apply(lambda x: round(float(x),2))
df_sim['reservesUSD'] = df_sim['reservesUSD'].apply(lambda x: round(float(x),2))
df_sim['liqUSD'] = df_sim['liqUSD'].apply(lambda x: round(float(x),2))
df_sim['treasury'] = df_sim['treasury'].apply(lambda x: round(float(x),2))
df_sim['liqRatio'] = df_sim['liqUSD'] / df_sim['treasury']
df_sim['marketcap'] = df_sim['marketcap'].apply(lambda x: round(float(x),2))
df_sim['supply'] = df_sim['supply'].apply(lambda x: round(float(x),2))
df_sim['askCapacity'] = df_sim['askCapacity'].apply(lambda x: round(float(x),2)) * df_sim['highWall']
df_sim['askCapacityTarget'] = df_sim['askCapacityTarget'].apply(lambda x: round(float(x),2)) * df_sim['highWall']
df_sim['bidCapacity'] = df_sim['bidCapacity'].apply(lambda x: round(float(x),2))
df_sim['bidCapacityTarget'] = df_sim['bidCapacityTarget'].apply(lambda x: round(float(x),2))
df_sims = df_sim[['marketcap', 'price', 'reservesUSD', 'netFlow', 'lowWall', 'lowCushion', 'highCushion', 'highWall', 'askCapacity', 'askCapacityTarget', 'bidCapacity', 'bidCapacityTarget']]
df_sims.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['price'], name='price (sims)'))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['lowWall'], name='lowWall (sims)', line=dict(color='grey', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['highWall'], name='highWall (sims)', line=dict(color='grey', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['lowCushion'], name='lowCushion (sims)', line=dict(color='grey', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['highCushion'], name='highCushion (sims)', line=dict(color='grey', dash='dot')))

fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['price'], name='price (testnet)'))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['lowWall'], name='lowWall (testnet)', line=dict(color='silver', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['highWall'], name='highWall (testnet)', line=dict(color='silver', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['lowCushion'], name='lowCushion (testnet)', line=dict(color='silver', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['highCushion'], name='highCushion (testnet)', line=dict(color='silver', dash='dot')))

fig.show()

In [ ]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['askCapacity'], name='askCapacity (sims)', line=dict(color='#009aff')))
fig2.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['askCapacityTarget'], name='askCapacityTarget (sims)', line=dict(color='grey', dash='dot')))

fig2.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['highCapacity'], name='askCapacity (testnet)', line=dict(color='#ff334e')))

fig2.show()

In [ ]:
fig3 = go.Figure()

fig3.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['bidCapacity'], name='bidCapacity (sims)', line=dict(color='#565bff')))
fig3.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['bidCapacityTarget'], name='bidCapacityTarget (sims)', line=dict(color='grey', dash='dot')))

fig3.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['lowCapacity'], name='bidCapacity (testnet)', line=dict(color='#ff59a2')))

fig3.show()

In [ ]:
fig4 = pd.DataFrame()
fig4['liqRatio (sims)'] = df_sim['liqRatio']
fig4['sliqRatio (testnet)'] = df_testnet2['liqRatio'][:-1]
fig4.plot()

In [ ]:
fig5 = pd.DataFrame()
fig5['supply (sims)'] = df_sim['supply']
fig5['supply (testnet)'] = df_testnet2['supply'][:-1]
fig5.plot()